In [1]:
### THROUGHOUT:
### - meaningful variable names
### - thorough comments that explain the code
### - general housekeeping
### - if possible, reorder columns
### - if possible, bypass writing XML; if cannot, make more meaningful name/location
### - FULL STOP: commit and submit pull request
### - reimplement core steps as functions: start with 2 big ones and try to do subfunctions from there
### - write in some broader level narrative using markdown cells
### - work on doing some plotting in R - ideally take advantage of SoS and pass dataframe directly to an R cell, but if we need to use an intermediate file then thats okay too.
### - calculate some summary statistics in R too. We have been focused on assembly length, but understanding distributions of other attributes is meaningful as well
### - also just knowing the counts of genomes and very basic stuff is good too

In [2]:
# importing the required modules 
import csv 
import requests 
import xml.etree.ElementTree as ET
import pandas as pd

In [3]:
# url for all assemblies
url = 'https://www.ebi.ac.uk/ena/data/view/Taxon:8782&portal=assembly&subtree=true&display=xml'
print(url)

https://www.ebi.ac.uk/ena/data/view/Taxon:8782&portal=assembly&subtree=true&display=xml


In [4]:
# creating HTTP response object from given url 
resp = requests.get(url) 

In [5]:
# saving the xml file 
with open('BGS.xml', 'wb') as f: 
    f.write(resp.content)

In [6]:
# create element tree object 
tree = ET.parse('BGS.xml') 

In [7]:
# get root element 
root = tree.getroot() 

In [8]:
#Retreiving all GCA ids through parsing and word search and saving to a dictionary.
GCA=[]
for node in tree.findall('.//IDENTIFIERS'):
    for snode in node.getchildren():
        if snode.tag == "PRIMARY_ID":
            word=snode.text
            if (word.find('GCA') != -1): 
                print (word) 
                GCA.append(word)

GCA_000002315.5
GCA_000146605.3
GCA_000151805.2
GCA_000238935.1
GCA_000247815.2
GCA_000277835.1
GCA_000331425.1
GCA_000332375.1
GCA_000337935.2
GCA_000337955.1
GCA_000337975.1
GCA_000355885.1
GCA_000385455.1
GCA_000400545.1
GCA_000400695.1
GCA_000511605.2
GCA_000534875.1
GCA_000586395.1
GCA_000599465.2
GCA_000599485.1
GCA_000687185.1
GCA_000687205.1
GCA_000687265.1
GCA_000687285.1
GCA_000687375.1
GCA_000690535.1
GCA_000690715.1
GCA_000690775.1
GCA_000690835.1
GCA_000690875.1
GCA_000691405.1
GCA_000691785.1
GCA_000691845.1
GCA_000691975.1
GCA_000692015.2
GCA_000692075.1
GCA_000695195.1
GCA_000695765.1
GCA_000695815.1
GCA_000696035.1
GCA_000696875.1
GCA_000698965.1
GCA_000699005.1
GCA_000699085.1
GCA_000699105.1
GCA_000699145.1
GCA_000699245.1
GCA_000699545.1
GCA_000699945.1
GCA_000700745.1
GCA_000703405.1
GCA_000705375.2
GCA_000708025.2
GCA_000708225.1
GCA_000708925.1
GCA_000709325.1
GCA_000709365.1
GCA_000709895.1
GCA_000710305.1
GCA_000737465.1
GCA_000738735.2
GCA_000747805.1
GCA_0007

/Users/cthillairajah17/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.
  after removing the cwd from sys.path.


In [10]:
# Retreiving xml for each assembly.
# This section iterates each GCA id retreived in the previous step into a url designe to query 
# the EBI database for the matching assembly metadata. The resulting xmle is then run 
# through the steps needed to use ET while remaining in the for loop.
dict=[]
for i in range(0, len(GCA)):
        
    url = 'https://www.ebi.ac.uk/ena/data/view/'+GCA[i]+'&display=xml'

    resp = requests.get(url) 
    with open('BGS.xml', 'wb') as f: 
        f.write(resp.content)
    tree = ET.parse('BGS.xml') 
    root = tree.getroot() 
    
    # Retreiving GCA id and sample id and saving to dictionary
    
    ID=[]
    for node in tree.findall('.//IDENTIFIERS'):
        for snode in node.getchildren():
           # print(snode.tag, snode.text)
            if snode.tag == "PRIMARY_ID":
                ID.append(snode.text)
   
    gca_id = ID[0] # The GCA id
    sample_id = ID[1] #The sample acession
    
    dict.append((i, 'gca_id', gca_id))
    dict.append((i, 'sample_id', sample_id))
    
    # Retreiving the taxon id and scientific name and saving to dictionary.
    taxon=[]
    for node in tree.findall('.//TAXON'):
        for snode in node.getchildren():
            taxon.append(snode.text)
   
    taxon_id = taxon[0] # The taxon id
    sname = taxon[1] # The scientific name
    

    dict.append((i, 'taxon_id', taxon_id))
    dict.append((i, 'sname_id', sname))
    
    # Retreiving all metadata fields by parsing all Assembly_Attribute subnodes and retreving TAGs and VALUEs
    
    tags=[]
    for node in tree.findall('.//ASSEMBLY_ATTRIBUTE'):
        for snode in node.getchildren():
            if snode.tag == "TAG":
                key=snode.text
                tags.append(key)
    

    values=[]            
    for node in tree.findall('.//ASSEMBLY_ATTRIBUTE'):
        for snode in node.getchildren():
            if snode.tag == "VALUE":
                value=snode.text
                values.append(value)
                
    # Saving data to a dictionary.
    for j in range(0, len(tags)):
        
        dict.append((i, tags[j], values[j]))
print(dict)

                
    
    
    

/Users/cthillairajah17/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: DeprecationWarning: This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.
/Users/cthillairajah17/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:34: DeprecationWarning: This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.
/Users/cthillairajah17/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:48: DeprecationWarning: This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.
/Users/cthillairajah17/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:56: DeprecationWarning: This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.


[(0, 'gca_id', 'GCA_000002315.5'), (0, 'sample_id', 'SAMN02981218'), (0, 'taxon_id', '9031'), (0, 'sname_id', 'Gallus gallus'), (0, 'total-length', '1065348650'), (0, 'ungapped-length', '1055564190'), (0, 'n50', '20785086'), (0, 'spanned-gaps', '878'), (0, 'unspanned-gaps', '68'), (0, 'scaffold-count', '524'), (0, 'count-contig', '1402'), (0, 'contig-n50', '17655422'), (0, 'contig-L50', '19'), (0, 'contig-n75', '9688072'), (0, 'contig-n90', '2759626'), (0, 'scaf-L50', '12'), (0, 'scaf-n75', '12763716'), (0, 'scaf-n90', '6153034'), (0, 'replicon-count', '34'), (0, 'count-non-chromosome-replicon', '0'), (0, 'count-alt-loci-units', '0'), (0, 'count-regions', '0'), (0, 'count-patches', '0'), (0, 'ENA-LAST-UPDATED', '2018-12-05'), (1, 'gca_id', 'GCA_000146605.3'), (1, 'sample_id', 'SAMN02981253'), (1, 'taxon_id', '9103'), (1, 'sname_id', 'Meleagris gallopavo'), (1, 'total-length', '1128339136'), (1, 'ungapped-length', '1093044709'), (1, 'n50', '3801642'), (1, 'spanned-gaps', '62525'), (1, '

In [11]:
# Saving dictionary to a pandas data frame.
df = pd.DataFrame(dict)
print(df)

# Using pivot function to arrange data into indexed rows with the tags as columns.
piv=df.pivot(index=0, columns=1, values=2)
print(piv)

        0                              1                             2
0       0                         gca_id               GCA_000002315.5
1       0                      sample_id                  SAMN02981218
2       0                       taxon_id                          9031
3       0                       sname_id                 Gallus gallus
4       0                   total-length                    1065348650
5       0                ungapped-length                    1055564190
6       0                            n50                      20785086
7       0                   spanned-gaps                           878
8       0                 unspanned-gaps                            68
9       0                 scaffold-count                           524
10      0                   count-contig                          1402
11      0                     contig-n50                      17655422
12      0                     contig-L50                            19
13    

In [12]:
#converting DataFrame into \ seperated .csv file.
pd.DataFrame.to_csv(piv, index=False, sep="\t")

'ENA-LAST-UPDATED\tcontig-L50\tcontig-n50\tcontig-n75\tcontig-n90\tcount-alt-loci-units\tcount-contig\tcount-non-chromosome-replicon\tcount-patches\tcount-regions\tgca_id\tn50\treplicon-count\tsample_id\tscaf-L50\tscaf-n75\tscaf-n90\tscaffold-count\tsname_id\tspanned-gaps\ttaxon_id\ttotal-length\tungapped-length\tunspanned-gaps\n2018-12-05\t19\t17655422\t9688072\t2759626\t0\t1402\t0\t0\t0\tGCA_000002315.5\t20785086\t34\tSAMN02981218\t12\t12763716\t6153034\t524\tGallus gallus\t878\t9031\t1065348650\t1055564190\t68\n2017-12-08\t11556\t26671\t11137\t1126\t0\t296331\t1\t0\t0\tGCA_000146605.3\t3801642\t33\tSAMN02981253\t82\t826892\t2279\t233806\tMeleagris gallopavo\t62525\t9103\t1128339136\t1093044709\t2520\n2014-01-17\t8015\t38644\t10023\t2988\t0\t124805\t0\t0\t0\tGCA_000151805.2\t8236790\t35\tSAMN02981239\t37\t2365395\t8511\t37421\tTaeniopygia guttata\t87384\t59729\t1232118738\t1222847838\t326\n2013-12-03\t5478\t55633\t24998\t8720\t0\t70862\t0\t0\t0\tGCA_000238935.1\t10614383\t0\tSAMN0298

In [13]:
help(pd.DataFrame)

Help on class DataFrame in module pandas.core.frame:

class DataFrame(pandas.core.generic.NDFrame)
 |  DataFrame(data=None, index=None, columns=None, dtype=None, copy=False)
 |  
 |  Two-dimensional size-mutable, potentially heterogeneous tabular data
 |  structure with labeled axes (rows and columns). Arithmetic operations
 |  align on both row and column labels. Can be thought of as a dict-like
 |  container for Series objects. The primary pandas data structure.
 |  
 |  Parameters
 |  ----------
 |  data : ndarray (structured or homogeneous), Iterable, dict, or DataFrame
 |      Dict can contain Series, arrays, constants, or list-like objects
 |  
 |      .. versionchanged :: 0.23.0
 |         If data is a dict, argument order is maintained for Python 3.6
 |         and later.
 |  
 |  index : Index or array-like
 |      Index to use for resulting frame. Will default to RangeIndex if
 |      no indexing information part of input data and no index provided
 |  columns : Index or arra